In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/zacklarsen/Documents/Projects/quarto/docs':
  os.chdir(r'/Users/zacklarsen/Documents/Projects/quarto/docs')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/importlib/_bootstrap.py": 1694438322.7574778, "/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/importlib/_bootstrap_external.py": 1694438322.765277, "/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/codecs.py": 1694438319.762392, "/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/encodings/aliases.py": 1694438320.950249, "/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/encodings/__init__.py": 1694438320.9463947, "/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/encodings/utf_8.py": 1694438321.707507, "/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/abc.py": 1694438319.6848092, "/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/io.py": 1694438319.9214835, "/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/stat.py": 1694438320.154745, "/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/_collections_abc.py": 1694438319.6394823, "/Users/zacklarsen/mambaforge/envs/quarto/li

/var/folders/g7/822123tx4gn6ndvw2jgh5f140000gn/T/ipykernel_26357/2033971976.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



In [2]:
import polars as pl
import pandas as pd
import altair as alt
from vega_datasets import data as vega_data

In [3]:
data = vega_data.gapminder()

data2000 = pl.DataFrame(data).filter(pl.col("year") == 2000).to_pandas()

select_year = alt.selection_single(
    name='select',
    fields=['year'],
    bind=alt.binding_range(min=1955, max=2005, step=5)
)

alt.Chart(data).mark_point(filled=True).encode(
    alt.X('fertility:Q', title='Fertility', scale=alt.Scale(domain=[0,9])),
    alt.Y('life_expect:Q', title='Life Expectancy', scale=alt.Scale(domain=[0,90])),
    alt.Size('pop:Q', title='Population', scale=alt.Scale(domain=[0, 1200000000], range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending')
).add_selection(select_year).transform_filter(select_year)

/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning:

'selection_single' is deprecated.  Use 'selection_point'

/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning:

'add_selection' is deprecated. Use 'add_params' instead.

/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/site-packages/altair/utils/core.py:410: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.



alt.Chart(...)

In [4]:
source = vega_data.cars()

brush = alt.selection(type='interval')

points = alt.Chart(source).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(brush, 'Origin:N', alt.value('gray'))
).add_selection(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='Origin:N',
    color='Origin:N',
    x='count(Origin):Q'
).transform_filter(
    brush
)

alt.vconcat(points, bars, data=source)

/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning:

'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.

/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning:

'add_selection' is deprecated. Use 'add_params' instead.

/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/site-packages/altair/utils/core.py:410: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.

/Users/zacklarsen/mambaforge/envs/quarto/lib/python3.10/site-packages/altair/utils/core.py:410: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtyp

alt.VConcatChart(...)